In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Entrainment'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
#########################################

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
##############################
#SETUP

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
elif Np_str=='50e6':
    num_jobs=1200 #50M parcels
    num_slurm_jobs=150
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def InitiateArray(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data1['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel1['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g','A_c','A_g_Processed','A_c_Processed']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )
def InitiateArray_Job(parcel,out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data1['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g','A_c','A_g_Processed','A_c_Processed']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(num_jobs,job_id):
    total_elements=len(parcel1['xh']) #total num of variables
    
    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    # job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    # if job_id==0: job_id=1
    start_job, end_job = get_job_range(job_id)
    # print(f'\nstart_job = {start_job}, end_job = {end_job}')

    index_adjust=start_job
    return start_job,end_job,index_adjust
# num_jobs=1000; job_id=1
# [start_job,end_job,index_adjust] = StartJobArray(num_jobs,job_id)

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def GetArrays(start_job,end_job):
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['A_g', 'A_c']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    A_g, A_c = (data_dict[k] for k in var_names)
    
    # # #Making Time Matrix
    # Nt=len(data['time'])
    # T = np.broadcast_to(np.arange(Nt)[:, None], A_c.shape)  # shape: (Nt, p)
    
    # check_memory(globals())
    return A_g, A_c

In [ ]:
###########################################################################################################################################################################

In [ ]:
################################
#FUNCTION

In [ ]:
def extend_idxs(f,case):
    out=np.sort(np.add.outer(f, np.arange(case)).ravel())

    # #OLD METHOD (SLOW)
    # if np.any(f)==True:
    #     out=np.sort(np.concatenate([np.arange(idx, idx + case-1+1) for idx in f]))
    # else: 
    #     out=f
    return out

def find_sandwiched_patterns(changes, case):
    arr=changes
    
    window_size = case + 1  # e.g., for case=2, window_size = 3
    # The interior zeros count is (window_size - 2) which is case - 1
    pattern1 = np.array([-1] + [0]*(case - 1) + [1])
    pattern2 = np.array([1] + [0]*(case - 1) + [-1])
    # print(pattern1,pattern2)
    
    # Manually construct sliding windows
    windows = np.array([arr[i:i + window_size] for i in range(len(arr) - window_size + 1)])
    # print("Sliding windows:\n", windows) #TESTING
    
    #THE ALGORITHM
    turb_d=[]
    turb_e=[]
    count=0;max_iter=len(data1['time']);
    while np.any(((windows == pattern1) | (windows == pattern2)).all(axis=1)):
        count+=1; 
        if count>=max_iter: 
            print(count)
            break
        
        next_ind = np.where(((windows == pattern1) | (windows == pattern2)).all(axis=1))[0][0]
        
        if (windows[next_ind] == pattern1).all():
            turb_d.append(next_ind)
        elif (windows[next_ind] == pattern2).all(): 
            turb_e.append(next_ind) #append to list
    
        windows[0:next_ind+(case)+1,:] = 0 #removes from windows
    
    turb_d=np.array(turb_d,dtype=int); turb_e=np.array(turb_e,dtype=int)

    #EXTEND REST OF INDEXES TO PROCESS
    turb_d=extend_idxs(turb_d,case=case)
    turb_e=extend_idxs(turb_e,case=case)
    return turb_d,turb_e

In [ ]:
# TESTING
# changes = np.array([0,0,0,-1,1,0,0,-1,0,0,0,1,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=1) #<=1 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,1,0,0,-1,0,0,1,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=2) #<=2 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=3) #<=3 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=4) #<=4 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=5) #<=5 in a row timesteps are removed
# print("Case matches at indices:", a,b)

In [ ]:
###### (amount of time inside/outside of cloud to count as entrainment/detrainment)
mins_thresh=5 #5 mins
######

t_per_mins=1/((data1['time'][1]-data1['time'][0])/1e9/60).item() #timesteps per minute (<=1)
def get_changes(B):
    changes = np.diff(np.concatenate(([B[0]], B)))  # Add 0s to detect edges
    return changes
def PreProcessing(A,p):
    B = A[:,p] #UNCOMMENT WHEN NOT TESTING***

    # Find the changes in the array
    changes=get_changes(B)
    # print(f'B = {B}'); print(f'changes = {changes}') 

    #Determining the Case Number
    case=int(t_per_mins*mins_thresh) #UNCOMMENT WHEN NOT TESTING***
    
    if case>1:
        for case_ind in np.arange(case,0,-1): 
        # for case_ind in [case]:
            #Calling Algorithm and Correcting Parcel Data
            [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case_ind)
            B[turb_d]=1
            B[turb_e]=0     
            changes=get_changes(B)
            # print(B)
    elif case==1:
        #Calling Algorithm and Correcting Parcel Data
        [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case)
        B[turb_d]=1
        B[turb_e]=0
    return B

In [ ]:
#RUNNING
def Apply(A_g,A_c):
    A_g_Processed = A_g.copy()
    A_c_Processed = A_c.copy()
    for p in np.arange(Np):
        if np.mod(p,1e3)==0: print(f"{p}/{len(parcel['xh'])}")
        out1=PreProcessing(A_g_Processed,p); A_g_Processed[:,p]=out1
        out2=PreProcessing(A_c_Processed,p); A_c_Processed[:,p]=out2
    return A_g_Processed,A_c_Processed

#SAVING
def Save(parcel,A_g_Processed,A_c_Processed,job_id):
    dir2=dir+'Project_Algorithms/Entrainment/Processing_Out/'
    out_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
    vars=['A_g_Processed','A_c_Processed']
    InitiateArray_Job(parcel,out_file,vars,t_chunk_size=50,p_chunk_size=1000)
    
    with h5py.File(out_file, 'a') as f: 
        f['A_g_Processed'][:]=A_g_Processed
        f['A_c_Processed'][:]=A_c_Processed

In [ ]:
# #EXAMPLE TESTING #CASE COUNTDOWN
# B=np.array([1,1,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,1,1,1,0,0,0,1])
#                     #,#,#,        #,#,#       #,#,#
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=3)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=2)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=1)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

In [ ]:
# # TESTING
# # Case 1
# case=1
# B=np.array([1,0,1,1,0,0,1,0])
# B=np.array([1,0,1,1,0,1,0]) 
# B=np.array([1,0,1,1,0,1,0,1])

# B=np.array([1,0,1,0,1,1,1])
# B=np.array([1,0,1,0,1,0,0])
# B=np.array([0,1,0,1,0,0,0]) 
# B=np.array([0,1,0,1,0,1,1])

# # Case 2
# case=2
# B=np.array([1,1,1,0,0,1,1,1,0,0,0,1,1,0,0,0])
# B=np.array([1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0]) #101 should still get removed

# # Case 3
# case=3
# B=np.array([1,1,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,1])
# print(f'output =  {B}\n')
# p=1234; out=PreProcessing(p,updraft_type='cloudy') #TESTING
# print(f'output =  {out}\n')

# # Case 5
# case=5
# B=np.array([1,1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,1])
# print(f'output =  {B}\n')
# p=1234; out=PreProcessing(p,updraft_type='cloudy') #TESTING
# print(f'output =  {out}\n')

# # REAL CASE
# count_per_row = (A_c >= 1).sum(axis=0)
# where=np.where(count_per_row > 10)[0] # Find row indices where count is greater than 10
# # print(where)
# ind=12345; p=where[ind]; print(p) 

# print(A_c[:,p])
# out=PreProcessing(p,updraft_type='cloudy') #TESTING
# print(f'output =  {out}\n')

In [ ]:
####################################
#RUNNING

In [ ]:
# %%time
########################################
#RUNNING
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 ==0: print(f"current job_id = {job_id}\n")
    [start_job,end_job,index_adjust]=StartJobArray(num_jobs,job_id)
    print(f'Running on Parcels {start_job}-{end_job}')
    parcel=parcel1.isel(xh=slice(start_job,end_job)); Np=len(parcel['xh'])
    [A_g,A_c]=GetArrays(start_job,end_job)
    [A_g_Processed,A_c_Processed]=Apply(A_g,A_c)
    Save(parcel,A_g_Processed,A_c_Processed,job_id)

In [ ]:
#COMBINING JOB_ARRAYS AFTER RUNNING
########################################################################
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
recombine=True 

In [ ]:
# def Recombine(num_jobs):
#     dir2=dir+'Project_Algorithms/Entrainment/Processing_Out/'
#     dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'
#     out_file=dir3+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
    
#     vars=['A_g_Processed','A_c_Processed']
#     InitiateArray(out_file,vars,t_chunk_size=50,p_chunk_size=100_000)
    
#     with h5py.File(out_file, 'r+') as f_out:
        
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,5)==0: print(f"job_id = {job_id}")
#             [a,b,_] = StartJobArray(num_jobs,job_id)
        
#             in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][:,a:b]=f_in[var][:]

In [ ]:
def Recombine_Dask(): #*#*
    #DASK-ENABLED
    import glob
    import re
    import dask
    from dask.diagnostics import ProgressBar
    
    def recombine_func(in_files,out_file):
        # matching_files = sorted(glob.glob(in_files))
        matching_files = sorted(
        glob.glob(in_files),
        key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
    )
        
        print(f'recombining {len(matching_files)} files')
        print('Forming Dask Delayed Object')
        # out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_1',combine='nested',phony_dims='sort')
        if Np_str == '1e6':
            dim_1_length = 100_000
        elif Np_str == '50e6':
            dim_1_length = 500_000
        chunks={'phony_dim_0': -1, 'phony_dim_1':dim_1_length}
        out = xr.open_mfdataset(
            matching_files,
            engine='h5netcdf',
            concat_dim='phony_dim_1',
            combine='nested',
            phony_dims='sort',
            chunks=chunks,
            parallel=True
        ) 
        print('RUNNING')
        with ProgressBar():
            delayed_write=out.to_netcdf(out_file, engine='h5netcdf', compute=False)
            delayed_write.compute()

    ####################################################################################
    dir2=dir+'Project_Algorithms/Entrainment/Processing_Out/'
    dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'

    in_files=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_*.h5'
    out_file=dir3+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
    recombine_func(in_files,out_file)

In [ ]:
if recombine==True:
    # Recombine(num_jobs=num_jobs)
    Recombine_Dask()

In [ ]:
#TESTING
############################

In [ ]:
# #TESTING RECOMBINE_ORIGINAL vs RECOMBINE_DASK
# import xarray as xr

# # Path to the output file you just saved
# out_file1 = dir + f'Project_Algorithms/Entrainment/OUTPUT/processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
# out_file2 = dir + f'Project_Algorithms/Entrainment/OUTPUT/processed_binary_arrays_{res}_{t_res}_{Np_str}_TEST.h5'

# # Read it in using xarray
# ds1=xr.open_dataset(out_file1, phony_dims='sort',engine='h5netcdf')
# ds2 = xr.open_dataset(out_file2, engine='h5netcdf')
# one=ds1['A_c_Processed'].isel(phony_dim_1=slice(1000, 2000)).data
# two=ds2['A_c_Processed'].isel(phony_dim_1=slice(1000, 2000)).data
# np.all(one==two)

In [ ]:
# #TESTING FOR EQUALITY BETWEEN A_G and A_C
# t=9
# p=50650806
# start_job=p;end_job=p+1
# parcel=parcel1.isel(xh=slice(start_job,end_job)); Np=len(parcel['xh'])
# [A_g,A_c]=GetArrays(start_job,end_job)
# [A_g_Processed,A_c_Processed]=Apply(A_g,A_c)

# print( np.where((A_g==A_c)&(A_c!=0)) )
# print( A_g[508]==A_c[508] )

# print( A_g[508] )
# print( A_c[508] )
# print( A_g_Processed[508] )
# print( A_c_Processed[508] )

# #WHY IS BOTH A_g and A_c changed from 0 to 1 #ISSUE CASE IS 508
# print('A_g')
# print( A_g[500:515].T ) 
# print('A_g_Processed')
# print( A_g_Processed[500:515].T ) #==> LEAVES GENERAL UPDRAFT FOR LESS THAN 5 mins, so all 0s are turned to 1s
# print('\n')
# print('A_c')
# print( A_c[500:515].T )
# print('A_c_Processed')
# print( A_c_Processed[500:515].T ) #==> LEAVES CLOUDY UPDRAFT FOR LESS THAN 5 mins, so all 0s are turned to 1s

# #PRINT SAVE
# # A_g
# # [[0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]]
# # A_g_Processed
# # [[0 0 0 0 0 0 1 1 1 1 1 0 0 0 0]]


# # A_c
# # [[1 1 1 1 1 1 0 0 0 1 0 1 1 1 1]]
# # A_c_Processed
# # [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]

# #CONCLUSION: 
# #Parcel moves between General and Cloudy Updrafts under 5 minutes, so for both Thresholds, it will count as Turbulent Entrainment and fill in as both in General and Cloudy Updraft, this will not effect final entrainment and detrainment values, but will affect the results of Updraft-Transfer Entrainment

In [ ]:
# #WALL TIME TESTING
# TESTING
# 27s/jobs = x/100_000 jobs
# (100_000*27/10)/60**2 = 75 hours (QUITE LONG)

# 30s/1job = x/10_000jobs
# (10_000*30/1)/60**2 = 83 hours (QUITE LONG)

# (5*60)s/1job = x/1000jobs
# (10_000*(5*60)/1)/60**2 = 830 hours (QUITE LONG)
# ==> if we then split into 100 slurm jobs ==> 8 hours 

In [ ]:
# # ###COMPARING SINGLE JOB RUN (COMPARING TO VERSION1)
# # #SINGLE JOB
# # dir2=dir+'Project_Algorithms/Entrainment/job_out_3/'
# # in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_{job_id}.h5'

# # with h5py.File(in_file, 'r') as f_in: 
# #     out=f_in['A_c_Processed'][:]
# # np.where(out!=A_c_Processed)[0]

# #ALL JOBS
# a=0;b=10000

# dir2=dir+'Project_Algorithms/Entrainment/'
# in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(in_file, 'r') as f_in: 
#     out1=f_in['A_c_Processed'][a:b]
    
# dir2=dir+'Project_Algorithms/Entrainment/'
# in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_TEST.h5'
# with h5py.File(in_file, 'r') as f_in: 
#     out2=f_in['A_c_Processed'][a:b]
# np.where(out1!=out2)[0]

In [ ]:
# start_job=0;end_job=10000
# # Reading Back Data Later
# ##############
# def make_data_dict(in_file,var_names,read_type):
#     if read_type=='h5py':
#         with h5py.File(in_file, 'r') as f:
#             data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            
#     elif read_type=='xarray':
#         in_data = xr.open_dataset(
#             in_file,
#             engine='h5netcdf',
#             phony_dims='sort',
#             chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
#         )
#         data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
#     return data_dict

# # read_type='xarray'
# read_type='h5py'

In [ ]:
# import h5py
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'

# var_names = ['A_g', 'A_c']
# data_dict = make_data_dict(in_file,var_names,read_type)
# [A_g, A_c] = (data_dict[k] for k in var_names) #, W

# check_memory(globals())

In [ ]:
# #READING BACK IN
# dir2=dir+'Project_Algorithms/Entrainment/'
# in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'

# var_names = ['A_g_Processed', 'A_c_Processed']
# data_dict = make_data_dict(in_file,var_names,read_type)
# A_g_Processed, A_c_Processed = (data_dict[k] for k in var_names)
# check_memory(globals())

In [ ]:
# ind=290 #time index
# where=np.where(A_c[ind]!=A_c_Processed[ind])
# where #which parcels have differences

In [ ]:
# ind2=0 #choose one of those parcels
# # ind2+=1
# where2=where[0][ind2]
# print(A_c[:,where2])
# print(A_c_Processed[:,where2])

In [ ]:
# #PLUGGING INTO ALGORITHM TO SEE WHERE THINGS WENT WRONG
# B=A_c[:,where2]
# print(B)
# PreProcessing(p=where2,updraft_type='cloudy')

In [ ]:
# def extend_idxs(f,case):
#     out=np.sort(np.add.outer(f, np.arange(case)).ravel())

#     # #OLD METHOD (SLOW)
#     # if np.any(f)==True:
#     #     out=np.sort(np.concatenate([np.arange(idx, idx + case-1+1) for idx in f]))
#     # else: 
#     #     out=f
#     return out

# def find_sandwiched_patterns(changes, case):
#     arr=changes
    
#     window_size = case + 1  # e.g., for case=2, window_size = 3
#     # The interior zeros count is (window_size - 2) which is case - 1
#     pattern1 = np.array([-1] + [0]*(case - 1) + [1])
#     pattern2 = np.array([1] + [0]*(case - 1) + [-1])
#     # print(pattern1,pattern2)
    
#     # Manually construct sliding windows
#     windows = np.array([arr[i:i + window_size] for i in range(len(arr) - window_size + 1)])
#     # print("Sliding windows:\n", windows) #TESTING
    
#     #THE ALGORITHM
#     turb_d=[]
#     turb_e=[]
#     count=0;max_iter=len(data['time']);
#     while np.any(((windows == pattern1) | (windows == pattern2)).all(axis=1)):
#         count+=1; 
#         if count>=max_iter: 
#             print(count)
#             break
        
#         next_ind = np.where(((windows == pattern1) | (windows == pattern2)).all(axis=1))[0][0]
        
#         if (windows[next_ind] == pattern1).all():
#             turb_d.append(next_ind)
#         elif (windows[next_ind] == pattern2).all(): 
#             turb_e.append(next_ind) #append to list
    
#         windows[0:next_ind+(case)+1,:] = 0 #removes from windows
    
#     turb_d=np.array(turb_d,dtype=int); turb_e=np.array(turb_e,dtype=int)

#     #EXTEND REST OF INDEXES TO PROCESS
#     turb_d=extend_idxs(turb_d,case=case)
#     turb_e=extend_idxs(turb_e,case=case)
#     return turb_d,turb_e

In [ ]:
# ###### (amount of time inside/outside of cloud to count as entrainment/detrainment)
# mins_thresh=5 #5 mins
# ######

# def get_changes(B):
#     changes = np.diff(np.concatenate(([B[0]], B)))  # Add 0s to detect edges
#     return changes
# def PreProcessing(p,updraft_type):

#     if updraft_type=='general':
#         A=A_g.copy()
#     elif updraft_type=='cloudy':
#         A=A_c.copy()
#     # B = A[:,p] #UNCOMMENT WHEN NOT TESTING***

#     # Find the changes in the array
#     changes=get_changes(B)
#     # print(f'B = {B}'); print(f'changes = {changes}') 

#     #Determining the Case Number
#     t_per_mins=1/((data['time'][1]-data['time'][0])/1e9/60).item() #timesteps per minute (<=1)
#     case=int(t_per_mins*mins_thresh) #UNCOMMENT WHEN NOT TESTING***
    
#     if case>1:
#         for case_ind in np.arange(case,0,-1): 
#         # for case_ind in [case]:
#             #Calling Algorithm and Correcting Parcel Data
#             [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case_ind)
#             B[turb_d]=1
#             B[turb_e]=0     
#             changes=get_changes(B)
#             # print(B)
#     elif case==1:
#         #Calling Algorithm and Correcting Parcel Data
#         [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case)
#         B[turb_d]=1
#         B[turb_e]=0
#     return B